In [1]:
import sys
sys.path.append('/Users/user/meepc')
import numpy as np
import pandas as pd
from models import Hankel,Corrhankel,RobustPipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def calculate_fpr(y_actual, y_prediction):
    fp = 0
    tn = 0
    
    for i in range(len(y_actual)):
        if y_actual[i] == 0 and y_prediction[i] == 1:
            fp += 1
        elif y_actual[i] == 0 and y_prediction[i] == 0:
            tn += 1
    fpr= fp/(fp+tn)
    return fpr

In [3]:
hankel = Hankel()
corrhankel = Corrhankel() 
lag = 50
stride = 0.5

In [4]:
df1 = pd.read_csv('~/data/ctown/dataset03.csv')
df2 = pd.read_csv('~/data/ctown/dataset04.csv')

train_normal = pd.concat((df1,df2),axis=0,ignore_index=True)
train_attack = df2[df2['ATT_FLAG']==1]

In [5]:
sensors = [col for col in train_normal.columns if col not in ['DATETIME','ATT_FLAG', 'S_PU6','F_PU6','S_PU11','F_PU11']]

In [6]:
y_truth=train_normal.loc[:,'ATT_FLAG']

In [7]:
#attack indexes in original training data
temp = hankel.fit(np.array(y_truth) ,lag,stride)
attack_idx = np.any(temp>0,axis=0).astype(int)
attack_idx = np.where(attack_idx>0)[0]
attack_idx

array([418, 419, 420, 421, 430, 431, 432, 442, 443, 444, 445, 446, 462,
       463, 464, 465, 466, 467, 489, 490, 491, 492, 498, 499, 500, 501,
       502, 503, 506, 507, 508, 509, 510, 511], dtype=int64)

In [8]:
scaler = StandardScaler()
X_normal = pd.DataFrame(index=train_normal.index, columns=sensors, data=scaler.fit_transform(train_normal[sensors]))

In [9]:
#  train_attack[sensors]
X_attack = pd.DataFrame(index=train_attack.index, columns = sensors, data=scaler.fit_transform(train_attack[sensors])).reset_index().drop(columns=['index'])

In [10]:
corr_normal,nolag_normal = corrhankel.fit(X_normal.to_numpy(),lag,stride)

In [11]:
corr_attack,nolag_attack = corrhankel.fit(X_attack.to_numpy(),lag,stride)

In [12]:
df_test = pd.read_csv('~/data/ctown/test_dataset.csv')

# Epasad with 1 cluster (Normal data)

In [13]:
X_test = pd.DataFrame(index=df_test.index, columns=sensors, data=scaler.fit_transform(df_test[sensors]))
corr_test,nolag_test = corrhankel.fit(X_test.to_numpy(),lag,stride)
Y_test = df_test.loc[:,'ATT_FLAG']

In [14]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [15]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
fpr = []
cluster_indexes=[]
pca_counts=0
meepc_counts=0

for i,sens in enumerate(sensors):
    print("sensor -> ", sens)
    train_normal = X_normal.loc[:,sens].values
    model = RobustPipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,kscore_init='inertia',tune=False,alpha=0.08,corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,only_corr=True,y_truth=y_truth)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_normal.columns)).T)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    fpr.append(calculate_fpr(y_actual, y_predicted))

    #clustering
    cluster_indexes.append(model.cluster_idx)
    #pca

    pca_counts=max(pca_counts,model.total_pca_attacks)
    
    #meepc
    meepc_counts=max(meepc_counts,model.counts_meepc)
    
    

sensor ->  L_T1
------[CLUSTER] attack points found is 2 
------[PCA] attack points found is 16 
------[Meepc] attack points found is 8 
sensor ->  L_T2
------[CLUSTER] attack points found is 4 
------[PCA] attack points found is 16 
------[Meepc] attack points found is 8 
sensor ->  L_T3
------[CLUSTER] attack points found is 3 
------[PCA] attack points found is 16 
------[Meepc] attack points found is 3 
sensor ->  L_T4
------[CLUSTER] attack points found is 0 
------[PCA] attack points found is 16 
------[Meepc] attack points found is 7 
sensor ->  L_T5
------[CLUSTER] attack points found is 4 
------[PCA] attack points found is 16 
------[Meepc] attack points found is 3 
sensor ->  L_T6
------[CLUSTER] attack points found is 8 
------[PCA] attack points found is 16 
------[Meepc] attack points found is 8 
sensor ->  L_T7
------[CLUSTER] attack points found is 4 
------[PCA] attack points found is 16 
------[Meepc] attack points found is 4 
sensor ->  F_PU1
------[CLUSTER] attack p

In [16]:
attack_idx 

array([418, 419, 420, 421, 430, 431, 432, 442, 443, 444, 445, 446, 462,
       463, 464, 465, 466, 467, 489, 490, 491, 492, 498, 499, 500, 501,
       502, 503, 506, 507, 508, 509, 510, 511], dtype=int64)

In [17]:
cluster_inactive_idx = np.unique(np.concatenate(cluster_indexes))
commonidxs=np.intersect1d(attack_idx,cluster_inactive_idx)
percentage = len(commonidxs)/len(attack_idx)*100
print(percentage)
print(commonidxs)



97.05882352941177
[418 419 420 421 430 431 432 442 443 444 445 446 462 463 464 465 466 467
 489 490 491 492 498 499 500 501 502 503 507 508 509 510 511]


In [18]:
percentage = pca_counts/len(attack_idx)*100
print(percentage)



47.05882352941176


In [19]:
percentage = meepc_counts/len(attack_idx)*100
print(percentage) 

44.11764705882353


In [20]:
sensor_predicted = np.asarray(sensor_predicted)
y_predicted = np.any(sensor_predicted,axis=0).astype(int)
y_predicted

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0])

In [21]:
print("Accuracy ",accuracy_score(y_actual,y_predicted))
print("Precision ",precision_score(y_actual,y_predicted))
print("Recall ",recall_score(y_actual,y_predicted))
print("F1-score ",f1_score(y_actual,y_predicted))
print("False Positive Rate ",calculate_fpr(y_actual, y_predicted))

Accuracy  0.8536585365853658
Precision  0.95
Recall  0.6333333333333333
F1-score  0.7599999999999999
False Positive Rate  0.019230769230769232


In [22]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.6541588492808003 Median : 0.6341463414634146 Min : 0.6341463414634146 Max : 0.7439024390243902
precision - Mean: 0.4583333333333333 Median : 0.0 Min : 0.0 Max : 1.0
recall -    Mean: 0.05555555555555555 Median : 0.0 Min : 0.0 Max : 0.3
f1 -        Mean: 0.09287700488370387 Median : 0.0 Min : 0.0 Max : 0.4615384615384615


# Multiple clusters + No threshold tuning

In [23]:
cluster_accuracy = []
cluster_precision = []
cluster_recall = []
cluster_fscore = []
cluster_fpr = []
for k in range(2,5):
    print('-------------------------------------',k,'-------------------------------------')
    
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    fpr = []
    cluster_indexes=[]
    pca_counts=0
    meepc_counts=0
    for i,sens in enumerate(sensors):
        print(sens)
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = RobustPipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k=k,tune=False,kscore_init='inertia',alpha=0.08,corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,only_corr=True,y_truth=y_truth)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_normal.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted)) 
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        fpr.append(calculate_fpr(y_actual, y_predicted))

        cluster_indexes.append(model.cluster_idx)
        pca_counts=max(pca_counts,model.total_pca_attacks)
        meepc_counts=max(meepc_counts,model.counts_meepc)
    

    #cluster
    cluster_inactive_idx = np.unique(np.concatenate(cluster_indexes))
    commonidxs=np.intersect1d(attack_idx,cluster_inactive_idx)
    percentage = len(commonidxs)/len(attack_idx)*100
    print("Cluster - attack points detected {} %".format(percentage))
    #PCA
    
    percentage = pca_counts/len(attack_idx)*100
    print("PCA - attack points detected {} %".format(percentage))
    #Meepc
    percentage = meepc_counts/len(attack_idx)*100
    print("Meepc - attack points detected {} %".format(percentage),"/n")
            
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    cluster_accuracy.append(accuracy_score(y_actual, y_predicted))
    cluster_precision.append(precision_score(y_actual,y_predicted))
    cluster_recall.append(recall_score(y_actual,y_predicted))
    cluster_fscore.append(f1_score(y_actual,y_predicted))
    cluster_fpr.append(calculate_fpr(y_actual,y_predicted))
    print('------Number of Clusters: ',k,'-----------') 
    print("Accuracy ",cluster_accuracy[-1])
    print("Precision ",cluster_precision[-1])
    print("Recall ",cluster_recall[-1])
    print("F1-score ",cluster_fscore[-1])
    print("False Positive Rate ",cluster_fpr[-1],"/n") 
    

------------------------------------- 2 -------------------------------------
L_T1
------[CLUSTER] attack points found is 8 
------[PCA] attack points found is 15 
------[Meepc] attack points found is 9 
------[PCA] attack points found is 1 
------[Meepc] attack points found is 0 
L_T2
------[CLUSTER] attack points found is 3 
------[PCA] attack points found is 10 
------[Meepc] attack points found is 4 
------[PCA] attack points found is 6 
------[Meepc] attack points found is 2 
L_T3
------[CLUSTER] attack points found is 4 
------[PCA] attack points found is 1 
------[Meepc] attack points found is 0 
------[PCA] attack points found is 15 
------[Meepc] attack points found is 3 
L_T4
------[CLUSTER] attack points found is 0 
------[PCA] attack points found is 9 
------[Meepc] attack points found is 4 
------[PCA] attack points found is 7 
------[Meepc] attack points found is 2 
L_T5
------[CLUSTER] attack points found is 2 
------[PCA] attack points found is 8 
------[Meepc] attack p

In [24]:
print("Accuracy Scores: ",cluster_accuracy)
print("Precision Scores: ",cluster_precision)
print("Recall Scores: ",cluster_recall)
print("F1 Scores: ",cluster_fscore)
print("False Positive Rates: ",cluster_fpr)

Accuracy Scores:  [0.9024390243902439, 0.8414634146341463, 0.8658536585365854]
Precision Scores:  [0.9583333333333334, 0.9047619047619048, 0.9130434782608695]
Recall Scores:  [0.7666666666666667, 0.6333333333333333, 0.7]
F1 Scores:  [0.8518518518518519, 0.7450980392156863, 0.7924528301886793]
False Positive Rates:  [0.019230769230769232, 0.038461538461538464, 0.038461538461538464]
